In [ ]:
!python -m jupytools export -nb "08_efficient_net.ipynb" -o .

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# https://www.kaggle.com/tanlikesmath/rcic-fastai-starter

## Imports 

In [3]:
#export
from collections import OrderedDict
import json
import os
from os.path import dirname, join
from functools import reduce
from pdb import set_trace

import cv2 as cv
import jupytools
import jupytools.syspath
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt

from catalyst.utils import get_one_hot
import pretrainedmodels
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from visdom import Visdom

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
import contextlib, io
with contextlib.redirect_stderr(io.StringIO()):
    from basedir import ROOT, NUM_CLASSES
    from catalyst_visdom import BatchMetricsPlotCallback, EpochMetricsPlotCallback
    from dataset import load_data, build_stats_index, RxRxDataset
    from model.utils import freeze_all, unfreeze_layers
    from visual import rgb, six, show_1, show

In [4]:
#export
torch.set_default_tensor_type(torch.FloatTensor)

## Dataset Reader

In [5]:
#export
from augmentation import JoinChannels, SwapChannels, Resize, ToFloat, Rescale
from augmentation import VerticalFlip, HorizontalFlip, PixelStatsNorm, composer
from augmentation import AugmentedImages

In [6]:
#export
class RxRxImages(Dataset):
    def __init__(self, meta_df, img_dir, site=1, channels=(1, 2, 3, 4, 5, 6), 
                 open_image=PIL.Image.open, n_classes=NUM_CLASSES, train=True):
        
        self.records = meta_df.to_records(index=False)
        self.img_dir = img_dir
        self.site = site
        self.channels = channels
        self.n = len(self.records)
        self.open_image = open_image
        self.n_classes = n_classes
        self.train = train
        
    def _get_image_path(self, index, channel):
        r = self.records[index]
        exp, plate, well = r.experiment, r.plate, r.well
        subdir = 'train' if self.train else 'test'
        path = f'{self.img_dir}/{subdir}/{exp}/Plate{plate}/{well}_s{self.site}_w{channel}.png'
        return path
    
    def __getitem__(self, index):
        paths = [self._get_image_path(index, ch) for ch in self.channels]
        img = np.stack([self.open_image(p) for p in paths])
        img = img.astype(np.float32)
        img = img.transpose(1, 2, 0)
        r = self.records[index]
        if self.train:
            sirna = r.sirna
            target = int(sirna)
            onehot = get_one_hot(target, num_classes=self.n_classes)
            return {'features': img, 'targets': target, 'targets_one_hot': onehot}
        else:
            id_code = r.id_code
            return {'features': img, 'id_code': id_code}
    
    def __len__(self): 
        return self.n

## Pipeline Preparation

In [7]:
#export
from split import StratifiedSplit
splitter = StratifiedSplit()
trn_df, val_df = splitter(pd.read_csv(ROOT/'train.csv')) 
tst_df = pd.read_csv(ROOT/'test.csv')

In [10]:
#export
sz = 512
trn_ds = AugmentedImages(
    ds=RxRxImages(trn_df, ROOT),
    tr=composer([
        #VerticalFlip(p=0.25),
        #HorizontalFlip(p=0.25),
    ], resize=sz)
)
val_ds = AugmentedImages(ds=RxRxImages(val_df, ROOT), tr=composer(resize=sz))
tst_ds = AugmentedImages(ds=RxRxImages(tst_df, ROOT, train=False), tr=composer(resize=sz))

In [11]:
#export
def new_loader(ds, bs, shuffle=True, num_workers=12):
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, num_workers=num_workers)

## Model

In [12]:
#export
def resnet18(head_fn=None, n_classes=NUM_CLASSES):
    model_fn = pretrainedmodels.__dict__['resnet18']
    model = model_fn(num_classes=1000, pretrained='imagenet')
    feat_dim = model.last_linear.in_features
    if head_fn is None:
        model.last_linear = nn.Linear(feat_dim, n_classes)
    else:
        model.last_linear = head_fn(feat_dim)
    new_conv = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
    conv1 = model.conv1.weight
    with torch.no_grad():
        new_conv.weight[:, :] = torch.stack([torch.mean(conv1, 1)]*6, dim=1)
    model.conv1 = new_conv
    return model

In [13]:
#export
def freeze_all(model):
    for name, child in model.named_children():
        print('Freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = False

In [14]:
#export
def unfreeze_all(model):
    for name, child in model.named_children():
        print('Un-freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = True

In [15]:
#export
def unfreeze(model, names):
    for name, child in model.named_children():
        if name not in names:
            continue
        print('Un-freezing layer:', name)
        for param in child.parameters():
            param.requires_grad = True

## Train

In [ ]:
#export
model = resnet18()
freeze_all(model)

In [ ]:
#export
from visdom import Visdom

In [ ]:
#export
class RollingLoss:
    def __init__(self, smooth=0.98):
        self.smooth = smooth
        self.prev = 0
    def __call__(self, curr, batch_no):
        a = self.smooth
        avg_loss = a*self.prev + (1 - a)*curr
        debias_loss = avg_loss/(1 - a**batch_no)
        self.prev = avg_loss
        return debias_loss

In [ ]:
#export
def create_loaders(batch_size):
    trn_dl = new_loader(trn_ds, bs=batch_size, shuffle=True)
    val_dl = new_loader(val_ds, bs=batch_size, shuffle=False)
    return OrderedDict([('train', trn_dl), ('valid', val_dl)])

In [ ]:
#export
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda:1')

In [ ]:
#export
epochs = 1000
patience = 500

opt = torch.optim.AdamW(model.parameters(), lr=0.0003)
sched = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.95)
model = model.to(device)
rolling_loss = dict(train=RollingLoss(), valid=RollingLoss())
steps = dict(train=0, valid=0)

trials = 0
best_metric = -np.inf
history = []
stop = False

vis = Visdom(server='0.0.0.0', port=9090,
             username=os.environ['VISDOM_USERNAME'],
             password=os.environ['VISDOM_PASSWORD'])

loaders = create_loaders(batch_size=32)

for epoch in range(1, epochs+1):
    print(f'Epoch [{epoch}/{epochs}]')
    
    if epoch == 1:
        unfreeze_layers(model, ['last_linear'])
    elif epoch == 3:
        unfreeze_all(model)
    
    curr_lr = opt.param_groups[0]['lr']
    vis.line(X=[epoch], Y=[curr_lr], win='lr', name='lr', update='append')    
    iteration = dict(epoch=epoch, train_loss=list(), valid_loss=list())
    print(f"Current learning rate: {opt.param_groups[0]['lr']:.2E}")
    
    for name, loader in loaders.items():
        is_training = name == 'train'
        count = 0
        metric = 0.0
        
        with torch.set_grad_enabled(is_training):
            for batch in loader:
                steps[name] += 1
                opt.zero_grad()
                x = batch['features'].to(device)
                y = batch['targets'].to(device)
                out = model(x)
                
                if is_training:
                    loss = loss_fn(out, y)
                    loss.backward()
                    opt.step()
                
                avg_loss = rolling_loss[name](loss.item(), steps[name])
                iteration[f'{name}_loss'].append(avg_loss)
                y_pred = out.softmax(dim=1).argmax(dim=1)
                acc = (y_pred == y).float().mean().item()
                metric += acc
                count += len(batch)
                vis.line(X=[steps[name]], Y=[avg_loss], name=f'{name}_loss', 
                         win=f'{name}_loss', update='append', 
                         opts=dict(title=f'Running Loss [{name}]'))
        
        metric /= count
        iteration[f'{name}_acc'] = metric
        vis.line(X=[epoch], Y=[avg_loss], name=f'{name}', win='avg_loss',
                 update='append', opts=dict(title='Average Epoch Loss'))
        vis.line(X=[epoch], Y=[metric], name=f'{name}', win='accuracy', 
                 update='append', opts=dict(title=f'Accuracy'))
        
        last_loss = iteration[f'{name}_loss'][-1]
        
        print(f'{name} metrics: accuracy={metric:2.3%}, loss={last_loss:.4f}')
          
        if is_training:
            sched.step()
          
        else:
            # sched.step(metric)
            if metric > best_metric:
                trials = 0
                best_metric = metric
                print('Score improved!')
                torch.save(model.state_dict(), f'train.{epoch}.pth')
            else:
                trials += 1
                if trials >= patience:
                    stop = True
                    break
    
    history.append(iteration)
    
    print('-' * 80)
    
    if stop:
        print(f'Early stopping on epoch: {epoch}')
        break

torch.save(history, 'history.pth')

## Test

In [46]:
device = torch.device('cuda:0')
model = resnet18()
model = model.to(device)
model.load_state_dict(torch.load('train.9.pth'))
_ = model.eval()

In [50]:
with torch.no_grad():
    test_dl = new_loader(tst_ds, bs=64)
    preds = {}
    for batch in tqdm(test_dl):
        out = model(batch['features'].to(device))
        y_hat = out.argmax(dim=-1).cpu().numpy()
        preds.update(dict(zip(batch['id_code'], y_hat.astype(int))))

In [53]:
preds_df = pd.DataFrame([
    {'id_code': id_code, 'sirna': sirna} 
    for id_code, sirna in preds.items()])
preds_df.head(3)

,id_code,sirna
0,RPE-09_3_G13,1059
1,RPE-11_3_L03,83
2,HEPG2-09_4_J14,262


In [54]:
len(preds_df)

19897

In [56]:
preds_df.to_csv('submit.csv', index=False, columns=['id_code', 'sirna'])

In [41]:
submit = pd.read_csv(ROOT/'test.csv')
submit['sirna'] = preds.astype(int)
submit.to_csv('submit.csv', index=False, columns=['id_code','sirna'])

In [42]:
len(submit)

19897

In [57]:
from IPython.display import FileLink
FileLink('submit.csv')

/home/ck/code/tasks/protein/submit.csv